## This document demonstrates how to import the python caliper module. The requirements for the Caliperpy package is listed below, copied and pasted directly from the TransCAD 9.0 GISDK help manual:

### TransCAD 9.0 Caliperpy installation instructions
Installing the Caliper Python Module
To program TransCAD in Python, you need:

TransCAD Version 9 or later
A Windows distribution of Python, version 3.x. At the time of writing, the latest Python version was 3.8. For example:
Anaconda Python for Windows, download at https://www.anaconda.com/download/
ActiveState Python, download at http://www.activestate.com/products/activepython
The Caliper Python Installer: C:\Program Files\TransCAD 2020\GISDK\Samples\Python\CaliperPy-v[yyyy-mm].win-amd64.exe
 

You can run the Caliper Python Installer once to install the CaliperPy package globally in your local Python program site-package folder.

 

The CaliperPy module use DCOM as a connection between TransCAD and Python. That means that DCOM must be enabled on your computer, and that the security settings for the DCOM objects "TransCAD.AutomationServer" and "TransCAD.AutomationServer (secondary)" must allow launch and access by users of your computer.

In [49]:
import caliper, caliperpy, traceback, sys
import numpy as np
import pandas as pd
import pandas as pd
from siuba import *
from siuba import group_by, summarize, _
from plotnine import ggplot, aes, geom_line

#git instructions - open terminal in jupyternotebook
## cd "C:/Users/JacobFo/OneDrive - City of Durham/Python/TRMG2"

In [50]:
#folder <- (r"C:/Users/JacobFo/TRMG2_v0.1/scenarios/base_2016/input/se_data/")
path = "c:/Users/JacobFo/TRMG2_v0.1/scenarios/base_2016/input/se_data/"
inpFname =  os.path.join(path, "se_2020.bin")
se_2020 = dk.GetDataFrameFromBin(inpFname)
print( "se_2020 shape: ", amar_df.shape )

se_2020 shape:  (3189, 45)


In [51]:
se_2020.head()

,CK for Gaps (TBD),TAZ Chk (TBD),COUNTY,TAZ,Type,HH,HH_POP,Median_Inc,Pct_Worker,Pct_Child,...,ParkCostU,ParkCostO,EmpSpaces,StudSpaces,OtherSpaces,Enplanements,AWDT,PCTCV,PCTAUTOEE,PCTCVEE
0,None,1,Chatham,1,Internal,327,869,None,None,None,...,None,None,None,None,None,0,None,None,None,None
1,None,2,Chatham,2,Internal,385,986,None,None,None,...,None,None,None,None,None,0,None,None,None,None
2,None,3,Chatham,3,Internal,591,1445,None,None,None,...,None,None,None,None,None,0,None,None,None,None
3,None,4,Chatham,4,Internal,599,1314,None,None,None,...,None,None,None,None,None,0,None,None,None,None
4,None,5,Chatham,5,Internal,203,433,None,None,None,...,None,None,None,None,None,0,None,None,None,None


In [52]:
county_sum = (
se_2020
    >>group_by(_.COUNTY)
    >>summarize(HH = _.HH.sum())

)

county_sum

,COUNTY,HH
0,Alamance New,12393
1,Chatham,21989
2,Durham,134651
3,Durham GAP,0
4,Franklin,22780
5,GAP,0
6,Granville,12981
7,Harnet Renum,14823
8,Johnston,67490
9,Johnston New,4966
